In [9]:
import os
import pandas as pd
from pathlib import Path

# 1️⃣ Ruta a la carpeta "data/processed"
base_path = Path.cwd().parent / "data/processed"

# 2️⃣ Buscar archivos que contengan 'consulta_05' (también comprimidos .gz)
csv_files = []
for pattern in ("*consulta_05*.csv", "*consulta_05*.csv.gz"):
    csv_files.extend(base_path.glob(pattern))

if not csv_files:
    raise FileNotFoundError(f"No se encontraron archivos CSV con 'consulta_05' en {base_path}")

# 3️⃣ Seleccionar el CSV más reciente por fecha de modificación
latest_csv = max(csv_files, key=os.path.getmtime)
print(f"📂 Cargando archivo más reciente: {latest_csv.name}")

# 4️⃣ Cargarlo en Pandas (compression se infiere por extensión)
df = pd.read_csv(latest_csv, low_memory=False)

# 5️⃣ Vista rápida de datos
print(f"✅ Archivo cargado con {df.shape[0]:,} filas y {df.shape[1]} columnas.")
print(df.head())

# Una por línea
print(*df.columns, sep="\n")


📂 Cargando archivo más reciente: consulta_05_2025-09-13_14-03_v01.csv
✅ Archivo cargado con 25,621 filas y 62 columnas.
                              device_id  SerialNumber  ...       lon  fecha_natural
0  61108548-c3f9-43bc-90aa-bd443c4879ad      70014336  ... -6.434826     2025-07-16
1  0de8dd0b-c699-464d-9558-c2265367cf21      70014324  ... -6.440487     2025-07-16
2  8036c30b-4306-48c4-993f-8379666a6fa9      70014328  ... -6.434713     2025-07-16
3  cfcd490e-2356-40fb-8efd-9f7940052771      70014320  ... -6.434924     2025-07-16
4  efff585d-444a-43b8-9e2b-31a27d27016c      70014321  ... -6.432121     2025-07-16

[5 rows x 62 columns]
device_id
SerialNumber
Model
mensajes_esperados
mensajes_recibidos
mensajes_sin_gps
pct_recibidos_vs_esperados
pct_sin_gps_vs_esperados
pct_sin_gps_recibidos
ultimo_mensaje_recibido
ultima_posicion_gps_valida
ultima_posicion_geom
visto_ultima_vez
numero_reinicios
media_ttf
porcentaje_bateria
fecha_cambio_bateria
suma_total_uplinks
ranch_name
customer_

In [5]:
# Filtrar por ranch_name = "Daniel Arias"
df_consulta_ganaderia = df[df["ranch_name"] == "Daniel Arias González"].copy()

# Ver primeras filas para comprobar
print(df_consulta_ganaderia.head())


                              device_id  SerialNumber  ...       lon  fecha_natural
0  61108548-c3f9-43bc-90aa-bd443c4879ad      70014336  ... -6.434826     2025-07-16
1  0de8dd0b-c699-464d-9558-c2265367cf21      70014324  ... -6.440487     2025-07-16
2  8036c30b-4306-48c4-993f-8379666a6fa9      70014328  ... -6.434713     2025-07-16
3  cfcd490e-2356-40fb-8efd-9f7940052771      70014320  ... -6.434924     2025-07-16
4  efff585d-444a-43b8-9e2b-31a27d27016c      70014321  ... -6.432121     2025-07-16

[5 rows x 62 columns]


In [18]:
import pandas as pd

# ⬇️ Ajusta estas fechas a tu periodo de análisis. 
# Si las dejas en None, usará el rango presente en daily_by_animal.
fecha_ini = None  # p.ej. pd.to_datetime("2025-08-31").date()
fecha_fin = None  # p.ej. pd.to_datetime("2025-09-13").date()

dfd = daily_by_animal.copy()  # columnas: fecha_natural, device_id, km_dia, ...

# Filtrado por ventana (si se define)
if fecha_ini is not None and fecha_fin is not None:
    dfd = dfd[(dfd['fecha_natural'] >= fecha_ini) & (dfd['fecha_natural'] <= fecha_fin)]
else:
    fecha_ini = dfd['fecha_natural'].min()
    fecha_fin = dfd['fecha_natural'].max()

# Media de la ganadería por día (promedio entre animales por fecha)
herd_daily = (dfd.groupby('fecha_natural', as_index=False)['km_dia']
              .mean()
              .rename(columns={'km_dia':'km_promedio_dia'}))

# Media diaria del periodo (único número)
media_periodo_km_dia = herd_daily['km_promedio_dia'].mean()

print(f"Media diaria de km de la ganadería en el periodo {fecha_ini}–{fecha_fin}: {media_periodo_km_dia:.3f} km/día")
print(f"Días considerados: {len(herd_daily)}")


Media diaria de km de la ganadería en el periodo 2025-07-15–2025-09-13: 0.862 km/día
Días considerados: 61


In [19]:
import pandas as pd
from datetime import datetime

# ⬇️ Opcional: define ventana de análisis. Si los dejas en None, usa el rango del propio DF.
fecha_ini = None  # p.ej. "2025-08-31"
fecha_fin = None  # p.ej. "2025-09-13"

dfd = daily_by_animal.copy()

# Tipos y filtrado básico
dfd['fecha_natural'] = pd.to_datetime(dfd['fecha_natural']).dt.date
dfd = dfd.dropna(subset=['km_dia'])

# Ventana de análisis
if fecha_ini is not None and fecha_fin is not None:
    fecha_ini = pd.to_datetime(fecha_ini).date()
    fecha_fin = pd.to_datetime(fecha_fin).date()
else:
    fecha_ini = dfd['fecha_natural'].min()
    fecha_fin = dfd['fecha_natural'].max()

dfd = dfd[(dfd['fecha_natural'] >= fecha_ini) & (dfd['fecha_natural'] <= fecha_fin)]

# 1) Media diaria de la ganadería (promedio entre animales por día) y media del periodo (único valor)
herd_daily = (dfd.groupby('fecha_natural', as_index=False)['km_dia']
              .mean()
              .rename(columns={'km_dia': 'km_promedio_dia'}))

media_periodo_km_dia = herd_daily['km_promedio_dia'].mean()

print("=== Media diaria de km de la ganadería (periodo) ===")
print(f"{media_periodo_km_dia:.3f} km/día  | días: {len(herd_daily)}  | ventana: {fecha_ini}–{fecha_fin}")

# 2) Tabla por animal: media, mínimo, máximo (km/día) y nº de días
id_cols = [c for c in ['device_id', 'serial_number', 'animal_name'] if c in dfd.columns and c is not None]
if not id_cols:
    id_cols = ['device_id']  # respaldo

stats_por_animal = (dfd.groupby(id_cols, dropna=False)['km_dia']
    .agg(km_media='mean', km_min='min', km_max='max', dias='count')
    .reset_index())

# Redondeo
for col in ['km_media','km_min','km_max']:
    stats_por_animal[col] = stats_por_animal[col].round(3)

print("\n=== Tabla por animal (media, min, max km/día en el periodo) — Top 20 con menos movimiento ===")
print(stats_por_animal.sort_values('km_media', ascending=True).head(20).to_string(index=False))

# (Opcional) Guardar a CSV
# out_path = f"data/processed/actividad_por_animal_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
# stats_por_animal.to_csv(out_path, index=False)
# print("Guardado en:", out_path)


=== Media diaria de km de la ganadería (periodo) ===
0.862 km/día  | días: 61  | ventana: 2025-07-15–2025-09-13

=== Tabla por animal (media, min, max km/día en el periodo) — Top 20 con menos movimiento ===
                           device_id  serial_number animal_name  km_media  km_min  km_max  dias
efff585d-444a-43b8-9e2b-31a27d27016c       70014321       RUBIO     0.584   0.016   2.567    61
541d140b-46fc-4be9-8b28-cc1807800bd1       70014327    MARIECHA     0.623   0.000   1.943    61
244a7b73-e270-4a76-96ce-6c307daac840       70014332     PALMERA     0.746   0.086   2.589    60
61108548-c3f9-43bc-90aa-bd443c4879ad       70014336     MONTESA     0.752   0.003   1.775    61
03f65b27-8dd0-4f1c-97d8-aa8a4458015b       70014334      CIERVA     0.773   0.186   2.551    60
0de8dd0b-c699-464d-9558-c2265367cf21       70014324     QUIROSA     0.784   0.023   2.262    48
6f787b7d-d4e8-446a-adbd-98d2171c836f       70013774   MARAVILLA     0.848   0.371   2.678    15
957ea2b8-639c-411c-a4d4-8